# Hypothetical Document Embeddings
This notebook goes over how to use Hypothetical Document Embeddings (HyDE), as described in [this paper](http://boston.lti.cs.cmu.edu/luyug/HyDE/HyDE.pdf). 

At a high level, HyDE is an embedding technique that takes queries, generates a hypothetical answer, and then embeds that generated document and uses that as the final example. 

In order to use HyDE, we therefor need to provide a base embedding model, as well as an LLMChain that can be used to generate those documents. By default, the HyDE class comes with some default prompts to use (see the paper for more details on them), but we can also create our own.

In [1]:
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings, HypotheticalDocumentEmbedder
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
base_embeddings = OpenAIEmbeddings()
llm = OpenAI()

In [3]:
# Load with `web_search` prompt
embeddings = HypotheticalDocumentEmbedder.from_llm(llm, base_embeddings, "web_search")

In [4]:
# Now we can use it as any embedding class!
embeddings.embed_query("Where is the Taj Mahal?")

[-0.011378335766494274,
 -0.002466956852003932,
 -0.00924257654696703,
 0.002674342133104801,
 -0.0049524856731295586,
 0.01894325762987137,
 -0.03162162005901337,
 -0.004509857390075922,
 -0.003915559034794569,
 -0.012659791857004166,
 0.01431268360465765,
 -0.006908717565238476,
 0.011879774741828442,
 0.011613578535616398,
 -0.018188003450632095,
 0.0020119473338127136,
 0.021345213055610657,
 -0.018324196338653564,
 -0.004547000862658024,
 -0.023672880604863167,
 -0.017482275143265724,
 -0.007397775538265705,
 -0.002460766350850463,
 0.014065058901906013,
 -0.020651865750551224,
 0.011019280180335045,
 0.019289931282401085,
 -0.038604624569416046,
 0.003299593459814787,
 -0.0065434714779257774,
 -0.014238396659493446,
 -0.02525767683982849,
 -0.02850155532360077,
 -0.02921966463327408,
 -0.021989036351442337,
 0.01796514168381691,
 -0.004642955493181944,
 0.0007769211661070585,
 0.01270312536507845,
 0.010226882994174957,
 0.021419500932097435,
 0.011044043116271496,
 0.01145881321

## Multiple generations
We can also generate multiple documents and then combine the embeddings for those. By default, we combine those by taking the average. We can do this by changing the LLM we use to generate documents to return multiple things.

In [8]:
multi_llm = OpenAI(n=4, best_of=4)

In [9]:
embeddings = HypotheticalDocumentEmbedder.from_llm(multi_llm, base_embeddings, "web_search")

In [10]:
embeddings.embed_query("Where is the Taj Mahal?")

[-0.010920658009126782,
 -0.0037738183164037764,
 -0.009463549708016217,
 -0.00080768798943609,
 -0.0009967219521058723,
 0.019170645624399185,
 -0.031780639197677374,
 -0.004530174017418176,
 -0.0022659169626422226,
 -0.010150704067200422,
 0.012668190523982048,
 -0.0033580497838556767,
 0.00893957240623422,
 0.007129515637643635,
 -0.017932259012013674,
 -0.0009588234170223586,
 0.023374392185360193,
 -0.016797400312498212,
 -0.006108453846536577,
 -0.021799248177558184,
 -0.019662374863401055,
 -0.00721543375402689,
 -0.00018618947069626302,
 0.01452364306896925,
 -0.01949624763801694,
 0.011948794242925942,
 0.02303491299971938,
 -0.04114736244082451,
 0.0037539063196163625,
 -0.008685363772201526,
 -0.02058780938386917,
 -0.02550318045541644,
 -0.03720558062195778,
 -0.030802046414464712,
 -0.023233956657350063,
 0.017242338974028826,
 -0.01150246406905353,
 -0.0023517973750131205,
 0.013649874599650502,
 0.010926479466434103,
 0.02252459805458784,
 0.009908133186399937,
 0.004070

## Using our own prompts
Besides using preconfigured prompts, we can also easily construct our own prompts and use those in the LLMChain that is generating the documents. This can be useful if we know the domain our queries will be in, as we can condition the prompt to generate text more similar to that.

In the example below, let's condition it generate text about a state of the union address (because we will use that in the next example).

In [8]:
prompt_template = """Please answer the user's question about the most recent state of the union address
Question: {question}
Answer:"""
prompt = PromptTemplate(input_variables=["question"], template=prompt_template)
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [9]:
embeddings = HypotheticalDocumentEmbedder(llm_chain=llm_chain, base_embeddings=base_embeddings)

In [10]:
embeddings.embed_query("Where is the Taj Mahal?")

[-0.007121483329683542,
 -0.008868738077580929,
 -0.02119413949549198,
 0.0008152805385179818,
 0.009272435680031776,
 0.01757347397506237,
 -0.035096485167741776,
 -0.01201632060110569,
 -0.0032831993885338306,
 -0.013422954827547073,
 0.009814905002713203,
 -0.004973684437572956,
 0.019251342862844467,
 -0.019125187769532204,
 -0.014066348783671856,
 -0.005796849261969328,
 0.015744218602776527,
 -0.019541501998901367,
 0.010287988930940628,
 -0.012041551060974598,
 -0.013410339131951332,
 -0.00039640453178435564,
 -0.0032059289515018463,
 0.013094950467348099,
 -0.02926809713244438,
 0.014078964479267597,
 0.014545739628374577,
 -0.043347060680389404,
 7.761525921523571e-05,
 -0.0017756399465724826,
 -0.031185662373900414,
 -0.023717250674962997,
 -0.032724760472774506,
 -0.044406767934560776,
 -0.010798919014632702,
 -0.004532139748334885,
 -0.019882122054696083,
 -0.010906150564551353,
 0.01718239113688469,
 0.013650035485625267,
 0.03247245028614998,
 0.003187005640938878,
 -0.00

## Using HyDE
Now that we have HyDE, we can use it as we would any other embedding class! Here is using it to find similar passages in the state of the union example.

In [15]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

with open('../state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

In [16]:
docsearch = FAISS.from_texts(texts, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)

In [19]:
print(docs[0].page_content)

In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections. 

We cannot let this happen. 

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. 
